# Para Modelos de Clasificación

## Sobremuestreo

El sobremuestreo, en el contexto del aprendizaje automático y especialmente en el manejo de conjuntos de datos desbalanceados, es una técnica que se utiliza para aumentar el número de muestras en la clase minoritaria (la clase menos representada) con el fin de equilibrar mejor la distribución de clases en el conjunto de datos.

El sobremuestreo se realiza en varios pasos:
- Dividir el conjunto de datos de entrenamiento en observaciones negativas y positivas;
- Duplicar las observaciones positivas (las que tienen ocurrencias raras) varias veces;
- Crear una nueva muestra de entrenamiento basada en los datos obtenidos;
- Barajar los datos: preguntas idénticas que se suceden no ayudarán al entrenamiento.

In [ ]:
# Importamos las librerias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Importamos el conjunto de datos

data = pd.read_csv('D:/Tripleten/datasets/travel_insurance_us_preprocessed.csv')

# Definimos el objetivo y el los elementos para entrenar
target = data['Claim']
features = data.drop('Claim', axis=1)

# Dividimos el conjunto en training y validación
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345
)

# Declaramos una funcion para obtener el sobremuestreo del conjunto de entrenamiento
# Separamos los elementos en los cuales su target tiene 0 y 1
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    # Concatenamos no sin antes exponenciar los resultados para tener un conjunto mas robusto.
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)

    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345
    )

    return features_upsampled, target_upsampled

# Llamamos la funcion y recibimos dos variables
features_upsampled, target_upsampled = upsample(features_train, target_train, 10)

# Entrenamos nuestro modelo
model = LogisticRegression(random_state=12345, solver='liblinear')
model.fit(features_upsampled,target_upsampled)

#Predecimos y entregamos el resultado.
predicted_valid = model.predict(features_valid)
print('F1:', f1_score(target_valid, predicted_valid))

F1: 0.13688212927756654


## Submuestreo

El submuestreo, es una técnica utilizada para abordar el desequilibrio de clases en un conjunto de datos, El submuestreo implica reducir el número de ejemplos de las clases mayoritarias para igualar el número de ejemplos de las clases minoritarias. Esto se hace típicamente seleccionando aleatoriamente una muestra de los ejemplos de las clases mayoritarias para que coincida con el número de ejemplos de las clases minoritarias.

El submuestreo se realiza en varios pasos:
- Dividir el conjunto de datos de entrenamiento en observaciones negativas y positivas;
- Soltar al azar una parte de las observaciones negativas;
- Crear una nueva muestra de entrenamiento basada en los datos obtenidos después de la caída;
- Mezclar los datos. Asegúrate de que los datos positivos no sigan a los negativos: esto dificultará que los algoritmos aprendan.
- Para descartar de forma aleatoria algunos de los elementos de la tabla, usa la función sample(). Esta toma frac (de fracción) y devuelve elementos aleatorios en cantidades tales que su fracción en la tabla inicial sea igual a frac.


## SMOTE

Algoritmo que crea datos nuevos,

In [ ]:
# Importing the modules
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

# Importing the library
data = pd.read_csv('D:/Tripleten/datasets/travel_insurance_us_preprocessed.csv')

target = data['Claim']
features = data.drop('Claim', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345
)

#Defining a function 
def downsample(features, target, fraction):
    '''This function will downsample the zeros in features and target, the will concatenate the results with the ones and will return a shuffle df'''
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    # Concatenating samples
    features_downsampled = pd.concat([features_zeros.sample(frac=0.1, random_state=12345)]+[features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=0.1, random_state=12345)]+[target_ones])

    # shuffling the dataframes
    features_downsampled =  shuffle(features_downsampled, random_state=12345)
    target_downsampled = shuffle(target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

# Calling the function
features_downsampled, target_downsampled = downsample(
    features_train, target_train, 0.1
)

model = LogisticRegression(random_state = 12345, solver= 'liblinear' )
model.fit(features_downsampled,target_downsampled)
predicted_valid = model.predict(features_valid)

print('F1:', f1_score(target_valid, predicted_valid))

F1: 0.13333333333333333


# Para estandarizar (Regresión y Clasificación)

## Standard Scaler

The Standard Scaler standardize the numeric values. Applies for classification and regression algorithms. Makes the númeric values to have a mean of '0' an standard deviation of '1'.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

data = pd.read_csv('D:/Tripleten/datasets/travel_insurance_us.csv')

target = data['Claim']
features = data.drop('Claim', axis=1)
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345
)

numeric = ['Duration', 'Net Sales', 'Commission (in value)', 'Age']

scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])

print(features_train.head())


      Agency    Agency Type Distribution Channel              Product Name  \
33312    EPX  Travel Agency               Online         Cancellation Plan   
50154    EPX  Travel Agency               Online         Cancellation Plan   
26729    EPX  Travel Agency               Online         Cancellation Plan   
37842    EPX  Travel Agency               Online  2 way Comprehensive Plan   
23588    EPX  Travel Agency               Online  2 way Comprehensive Plan   

       Duration Destination  Net Sales  Commission (in value) Gender       Age  
33312  0.243485       CHINA  -0.513068              -0.500631    NaN -0.283389  
50154 -0.266879   SINGAPORE  -0.554403              -0.500631    NaN -0.283389  
26729 -0.329555    MALAYSIA  -0.637074              -0.500631    NaN -0.709982  
37842 -0.069896    THAILAND   0.065625              -0.500631    NaN -0.354488  
23588 -0.374324    MALAYSIA  -0.430398              -0.500631    NaN -0.425587  


In [26]:
# Importing the libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Importing the dataset
data = pd.read_csv('D:/Tripleten/datasets/travel_insurance_us.csv')

# Encoding categoric values into binaries
data_ohe = pd.get_dummies(data, drop_first=True, dummy_na=True)

features = data_ohe.drop(columns='Claim')
target = data_ohe['Claim']

# Creating a validation and training dataset
x_train, x_val, y_train, y_val = train_test_split(features, target, test_size= 0.25, random_state=12345)

# Extracting the columns with int and float values
numeric = features.select_dtypes(include=['int','float']).columns

# Escalating the numeric values
scaler = StandardScaler()
scaler.fit(x_train[numeric])

# Transforming and passing the values into the features and target
x_train[numeric] = scaler.transform(x_train[numeric])
x_val[numeric] = scaler.transform(x_val[numeric])

print(x_train.head())

       Duration  Net Sales  Commission (in value)       Age  Agency_ART  \
33312  0.243485  -0.513068              -0.500631 -0.283389       False   
50154 -0.266879  -0.554403              -0.500631 -0.283389       False   
26729 -0.329555  -0.637074              -0.500631 -0.709982       False   
37842 -0.069896   0.065625              -0.500631 -0.354488       False   
23588 -0.374324  -0.430398              -0.500631 -0.425587       False   

       Agency_C2B  Agency_CBH  Agency_CCR  Agency_CSR  Agency_CWT  ...  \
33312       False       False       False       False       False  ...   
50154       False       False       False       False       False  ...   
26729       False       False       False       False       False  ...   
37842       False       False       False       False       False  ...   
23588       False       False       False       False       False  ...   

       Destination_URUGUAY  Destination_UZBEKISTAN  Destination_VANUATU  \
33312                False   

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score

data = pd.read_csv('D:/Tripleten/datasets/travel_insurance_us_preprocessed.csv')

target = data['Claim']
features = data.drop('Claim', axis=1)

target_pred_constant = data['Claim'].replace(data['Claim'].values, 0)
print(target_pred_constant)

0        0
1        0
2        0
3        0
4        0
        ..
50655    0
50656    0
50657    0
50658    0
50659    0
Name: Claim, Length: 50660, dtype: int64


In [ ]:
# Importing libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Importing Data sets
data = pd.read_csv('/datasets/flights.csv')

# Using OHE for categoric column 'Airlines'
data_ohe = pd.get_dummies(data, drop_first= True)

# Defining target and features
target = data_ohe['Arrival Delay']
features = data_ohe.drop(['Arrival Delay'], axis=1)

# Slicing the dataset to receive a train and a valid dataset.
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12345
)

# Defining the columns to be used in the scaler class
numeric = features.select_dtypes(include=['int', 'float']).columns
#numeric = ['Day', 'Day Of Week', 'Origin Airport Delay Rate',
#       'Destination Airport Delay Rate', 'Scheduled Time', 'Distance',
#       'Scheduled Departure Hour', 'Scheduled Departure Minute']

# Using the StandardScaler class to transform the numeric values
scaler = StandardScaler()
scaler.fit(features_train[numeric])

features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])

print(features_train.shape)
print(features_valid.shape)

(58431, 22)
(19478, 22)
